СТАТИСТИКА ПО ИСПОЛНИТЕЛЮ

In [1429]:

# библиотеки и функции
import requests, json, urllib.parse
from statistics import mean, median
# конвертер времени
def convert_millis(millis: int) -> str:
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = (total_seconds // 60) % 60
    hours = (total_seconds // (60 * 60))
    return f"{hours}:{minutes}:{seconds}"



In [1430]:
base_url = 'https://itunes.apple.com/search'

artist = 'Metallica'

params = {
    'term':  artist,
    'limit': 250,
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

response = requests.get(f"{base_url}?{urllib.parse.urlencode(params)}")

data = json.loads(response.text)
data = data['results']

In [1431]:
import dateutil.parser
data = map(lambda x: {**x, 'releaseDate': dateutil.parser.isoparse(x['releaseDate'])}, data)
data = list(data)


In [1432]:
data = map(lambda x: {
    'artistId': x['artistId'],
    'collectionId': x['collectionId'],
    'trackId': x['trackId'],
    'artistName': x['artistName'],
    'collectionName': x['collectionCensoredName'],
    'trackName': x['trackCensoredName'],
    'collectionPrice': x['collectionPrice'],
    'trackPrice': x['trackPrice'],
    'releaseDate': x['releaseDate'],
    'collectionExplicitness': x['collectionExplicitness'],
    'trackExplicitness': x['trackExplicitness'],
    'discCount': x['discCount'],
    'discNumber': x['discNumber'],
    'trackCount': x['trackCount'],
    'trackNumber': x['trackNumber'],
    'trackTimeMillis': x['trackTimeMillis'],
    'country': x['country'],
    'currency': x['currency'],
    'primaryGenreName': x['primaryGenreName'],
    'isStreamable': x['isStreamable']
}, data)
data = list(data)
print(data[0])


{'artistId': 3996865, 'collectionId': 1572046434, 'trackId': 1572046436, 'artistName': 'Metallica', 'collectionName': 'Metallica (Remastered)', 'trackName': 'Enter Sandman', 'collectionPrice': 9.99, 'trackPrice': 1.29, 'releaseDate': datetime.datetime(1991, 7, 29, 12, 0, tzinfo=tzutc()), 'collectionExplicitness': 'notExplicit', 'trackExplicitness': 'notExplicit', 'discCount': 1, 'discNumber': 1, 'trackCount': 11, 'trackNumber': 1, 'trackTimeMillis': 331560, 'country': 'USA', 'currency': 'USD', 'primaryGenreName': 'Metal', 'isStreamable': True}


Название исполнителя

In [1433]:
print(set(map(lambda x: x['artistName'], data)))

{'Metallica', 'Metallica & San Francisco Symphony'}


In [1434]:
data = filter(lambda x: artist.lower() in x['artistName'].lower(), data)
data = list(data)

Количество песен

In [1435]:
chislo = lambda x: x['trackName']
chislo_songs = len(list(map(chislo, data)))
print('Count of songs:', chislo_songs)

Count of songs: 100


Популярный жанр

In [1436]:
primaryGenreName = max(list(map(lambda x: x['primaryGenreName'], data)))
print(primaryGenreName)

Metal


Количество альбомов

In [1437]:
from itertools import groupby

collection_id_extractor = lambda x: x['collectionId']

sorted_by_album = sorted(data, key=collection_id_extractor)
album_groups = {}
album_names = {}

for album_id, songs in groupby(sorted_by_album, key=collection_id_extractor):
    songs = list(songs)
    album_groups[album_id] = songs 
    album_names[album_id] = songs[0]['collectionName']

del sorted_by_album

print(list(album_names.values()))
print('count of albums:',len(set(list(map(lambda x: x['collectionId'], data)))))

['Beyond Magnetic - EP', 'I Disappear - Single', "Kill 'Em All (Remastered)", 'Ride the Lightning (Remastered)', 'Death Magnetic', 'Garage Inc.', 'Load', 'Metallica', 'Reload', 'S&M (Live)', 'Hardwired…To Self-Destruct (Deluxe Edition)', 'Master of Puppets (Remastered)', '...And Justice for All (Remastered)', 'S&M2 (Live)', 'Metallica (Remastered)']
count of albums: 15


Общая длительность песен

In [1438]:
time_extractor = lambda x: x['trackTimeMillis']

Duration_of_songs = sum(list(map(time_extractor, data)))
print('Duration of songs:', convert_millis(Duration_of_songs))


Duration of songs: 10:18:55


Средняя длительность песни

In [1439]:
AverageTime = int(mean(list(map(lambda x: x['trackTimeMillis'], data))))
print('Average time:',convert_millis( AverageTime))

Average time: 0:6:11


Медиана длительности песен

In [1440]:
Median_time = list(map(time_extractor, data))
print('Mediana of songs:', convert_millis(median(Median_time)))

Mediana of songs: 0.0:6.0:27.0


название самой долгой песни(
длительность самой долгой песни)


название самой короткой песни(
длительность самой короткой песни)

In [1441]:
time_extractor = lambda x: x['trackTimeMillis']
long_song = max(data, key=time_extractor)
short_song = min(data, key=time_extractor)
print('Longe song:', long_song['trackName'], convert_millis(long_song['trackTimeMillis']))

shortest_song = min(data, key=time_extractor)

print('Short song:', short_song['trackName'], convert_millis(short_song['trackTimeMillis']))

Longe song: Suicide & Redemption 0:9:57
Short song: Helpless (Live at Rasputin Music, Berkeley, CA - Apr. 16th, 2016) 0:3:8


Общая стоимость песен

In [1442]:
track_price = lambda x: x['trackPrice']
all_track_price= sum(list(map(track_price,data)))

print('all track price:', all_track_price,'$')


all track price: 129.00000000000028 $


Средняя стоимость песен

In [1443]:
track_price = lambda x: x['trackPrice']
all_track_price= mean(list(map(track_price,data)))
print('Average price of songs',all_track_price,'$')

Average price of songs 1.29 $


Медиана стоимости песен

In [1444]:
track_price = lambda x: x['trackPrice']
all_track_price= median(list(map(track_price,data)))
print('Median of track price',all_track_price,'$')

Median of track price 1.29 $


название самой дорогой песни
(стоимость самой дорогой песни)


название самой дешевой песни
(стоимость самой дешевой песни)

In [1445]:
track_price = lambda x: x['trackPrice']
first=min(data, key=track_price)
print('min track price:', first['trackName'], first['trackPrice'],'$')
second=max(data, key=track_price)
print('max track price:', second['trackName'], second['trackPrice'],'$')



min track price: Enter Sandman 1.29 $
max track price: Enter Sandman 1.29 $


СТАТИСТИКА ПО АЛЬБОМУ

In [1446]:
album_count = len( list(set(map(lambda x: x['collectionName'], data))));
long_song = max(data, key=time_extractor)
short_song = min(data, key=time_extractor)
timeCount = sum(list(map(lambda x: x['trackTimeMillis'], data)))
AverageTime = int(mean(list(map(lambda x: x['trackTimeMillis'], data))))
allTrackPrice = sum(list(map(lambda x: x['trackPrice'], data)))
averageTrackPrice = mean(list(map(lambda x: x['trackPrice'], data)))
medianTrackPrice = median(list(map(lambda x: x['trackPrice'], data)))
track_price = lambda x: x['trackPrice']
first=min(data, key=track_price)	
second=max(data, key=track_price)
primaryGenreName= max(list(map(lambda x: x['primaryGenreName'], data)))
release_date_extractor = lambda x: x['releaseDate']
date = max(data, key=release_date_extractor)
Median_time = list(map(time_extractor, data))
print('========================')
print('Ну что,народ, погнали?')
print('========================')
for i in range( album_count ):
    album_name = list(set(map(lambda x: x['collectionName'], data )))[i]
    album_data = list(filter(lambda x:album_name in x['collectionName'], data ))
    print('Albom #',[i+1],':')
    print()
    print( 'Album name:', max( list(map(lambda x: x['collectionName'],data ))))
    print( 'Count of songs:', len(list(map(lambda x: x['trackName'],data ))))
    print('Primary genere:',primaryGenreName)
    print('Release date:', date['trackName'], date['releaseDate'])
    print('Average time:',convert_millis(AverageTime))
    print('Median songs:', convert_millis(median(Median_time)))
    print('All track price of album:',allTrackPrice,'$')
    print('Average track price of album:',averageTrackPrice,'$')
    print('Median track price of album:',medianTrackPrice,'$')
    print('min track price:', first['trackName'], first['trackPrice'],'$')
    print('max track price:', second['trackName'], second['trackPrice'],'$')
    print('Longe song:', long_song['trackName'], convert_millis(long_song['trackTimeMillis']))
    print('Short song:', short_song['trackName'], convert_millis(short_song['trackTimeMillis']))
    print()

Ну что,народ, погнали?
Albom # [1] :

Album name: S&M2 (Live)
Count of songs: 100
Primary genere: Metal
Release date: The Day That Never Comes (Live) 2020-08-28 12:00:00+00:00
Average time: 0:6:11
Median songs: 0.0:6.0:27.0
All track price of album: 129.00000000000028 $
Average track price of album: 1.29 $
Median track price of album: 1.29 $
min track price: Enter Sandman 1.29 $
max track price: Enter Sandman 1.29 $
Longe song: Suicide & Redemption 0:9:57
Short song: Helpless (Live at Rasputin Music, Berkeley, CA - Apr. 16th, 2016) 0:3:8

Albom # [2] :

Album name: S&M2 (Live)
Count of songs: 100
Primary genere: Metal
Release date: The Day That Never Comes (Live) 2020-08-28 12:00:00+00:00
Average time: 0:6:11
Median songs: 0.0:6.0:27.0
All track price of album: 129.00000000000028 $
Average track price of album: 1.29 $
Median track price of album: 1.29 $
min track price: Enter Sandman 1.29 $
max track price: Enter Sandman 1.29 $
Longe song: Suicide & Redemption 0:9:57
Short song: Helple